<center><img src='https://i.postimg.cc/TPR1n1rp/AI-Tech-PL-RGB.png' height="60"></center>

AI TECH - Akademia Innowacyjnych Zastosowań Technologii Cyfrowych. Programu Operacyjnego Polska Cyfrowa na lata 2014-2020
<hr>

<center><img src='https://i.postimg.cc/Gpq2KRQz/logotypy-aitech.jpg'></center>

<center>
Projekt współfinansowany ze środków Unii Europejskiej w ramach Europejskiego Funduszu Rozwoju Regionalnego 
Program Operacyjny Polska Cyfrowa na lata 2014-2020,
Oś Priorytetowa nr 3 "Cyfrowe kompetencje społeczeństwa" Działanie  nr 3.2 "Innowacyjne rozwiązania na rzecz aktywizacji cyfrowej" 
Tytuł projektu:  „Akademia Innowacyjnych Zastosowań Technologii Cyfrowych (AI Tech)”
</center>

# Lab 03: Imitation Learning

In this lab, we look into the problem of learning from expert demonstrations.

- Find a policy $\pi(a | s)$ that best imitates the expert policy $\pi^*(a | s)$ in the given environment.
- It's worth noting, that we don't need access to the environment rewards.

Major Imitation Learning techniques are:

1. Behavioural Cloning,
1. Imitation Learning via Interactive Demonstrator e.g. SMILe (Ross and Bagnell, 2010) or DAgger (Ross et al., 2011),
1. Inverse Reinforcement Learning -- out of scope of this lab.

We will solve the Ant problem, shown below, examining the first two approaches.

In [ ]:
#@title Mount your Google Drive

#@markdown Your work will be stored in a folder called `rl_lab_2022` by default.

#@markdown Run each section with Shift+Enter

#@markdown Double-click on section headers to show code.

import os
from google.colab import drive
drive.mount('/content/gdrive')

LAB_PATH = '/content/gdrive/MyDrive/rl_lab_2022/imitation_learning'
if not os.path.exists(LAB_PATH):
  %mkdir -p $LAB_PATH

MJC_PATH = '{}/mujoco'.format(LAB_PATH)
if not os.path.exists(MJC_PATH):
    %mkdir $MJC_PATH

Mounted at /content/gdrive


In [ ]:
#@title Install requirements

!apt -q update 
!apt install -q -y --no-install-recommends \
        build-essential \
        curl \
        git \
        gnupg2 \
        make \
        cmake \
        ffmpeg \
        swig \
        libz-dev \
        unzip \
        zlib1g-dev \
        libglfw3 \
        libglfw3-dev \
        libxrandr2 \
        libxinerama-dev \
        libxi6 \
        libxcursor-dev \
        libgl1-mesa-dev \
        libgl1-mesa-glx \
        libglew-dev \
        libosmesa6-dev \
        lsb-release \
        ack-grep \
        patchelf \
        wget \
        xpra \
        xserver-xorg-dev \
        xvfb \
        python-opengl \
        ffmpeg
!pip -q install gdown

# Installing dependencies for visualization
!apt-get -qq -y install libcusparse8.0 libnvrtc8.0 libnvtoolsext1 > /dev/null
!ln -snf /usr/lib/x86_64-linux-gnu/libnvrtc-builtins.so.8.0 /usr/lib/x86_64-linux-gnu/libnvrtc-builtins.so
!apt-get -qq -y install xvfb freeglut3-dev ffmpeg> /dev/null
!pip -q install -U gym==0.19
!pip -q install pyglet
!pip -q install pyopengl
!pip -q install pyvirtualdisplay

Get:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu focal InRelease
Get:4 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:5 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Get:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease [18.1 kB]
Get:7 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:8 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:9 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Get:10 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 Packages [3,167 kB]
Hit:11 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Get:12 http://archive.ubuntu.com/ubuntu focal-updates/restricted amd64 Packages [2,366 kB]
Get:13 http://archive.ubuntu.com/ubuntu focal-updates/universe amd64 Packag

In [ ]:
#@title Download MuJoCo

if not os.path.exists(f'{MJC_PATH}/mujoco210'):
    %cd $MJC_PATH
    !wget -q https://mujoco.org/download/mujoco210-linux-x86_64.tar.gz
    !tar -xzf mujoco210-linux-x86_64.tar.gz
    %rm mujoco210-linux-x86_64.tar.gz

/content/gdrive/MyDrive/rl_lab_2022/imitation_learning/mujoco


In [ ]:
#@title Install MuJoCo

import os

os.environ['LD_LIBRARY_PATH'] += ':{}/mujoco210/bin'.format(MJC_PATH)
os.environ['MUJOCO_PY_MUJOCO_PATH'] = '{}/mujoco210'.format(MJC_PATH)

# Installation on colab does not find *.so files in LD_LIBRARY_PATH,
# copy over manually instead.
!cp $MJC_PATH/mujoco210/bin/*.so /usr/lib/x86_64-linux-gnu/

In [ ]:
#@title Clone and install mujoco-py

if not os.path.exists(f'{MJC_PATH}/mujoco-py'):
    %cd $MJC_PATH
    !git clone https://github.com/openai/mujoco-py.git

%cd $MJC_PATH/mujoco-py
!git checkout f1312cceeeebbba17e78d5d77fbffa091eed9a3a # Tested version
%pip install -e .

# Compile at the first import
os.environ['LD_LIBRARY_PATH'] += ':/usr/lib/nvidia'
os.environ['LD_LIBRARY_PATH'] += ':/content/gdrive/MyDrive/rl_lab_2022/imitation_learning/mujoco/mujoco210/bin'
import mujoco_py

/content/gdrive/MyDrive/rl_lab_2022/imitation_learning/mujoco
Cloning into 'mujoco-py'...
remote: Enumerating objects: 2186, done.
remote: Counting objects: 100% (38/38), done.
remote: Compressing objects: 100% (36/36), done.
remote: Total 2186 (delta 9), reused 21 (delta 1), pack-reused 2148
Receiving objects: 100% (2186/2186), 5.70 MiB | 11.81 MiB/s, done.
Resolving deltas: 100% (1352/1352), done.
Updating files: 100% (200/200), done.
/content/gdrive/MyDrive/rl_lab_2022/imitation_learning/mujoco/mujoco-py
M	.dockerignore
M	.gitignore
M	mujoco_py/tests/test_substep.py
M	scripts/gen_wrappers.py
M	vendor/Xdummy-entrypoint
Note: switching to 'f1312cceeeebbba17e78d5d77fbffa091eed9a3a'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by switching back to a branch.

If you want to create a new branch to retain commits you create, you may
do so (now or late

INFO:root:running build_ext
INFO:root:building 'mujoco_py.cymj' extension
INFO:root:creating /content/gdrive/MyDrive/rl_lab_2022/imitation_learning/mujoco/mujoco-py/mujoco_py/generated/_pyxbld_2.1.2.14_310_linuxcpuextensionbuilder
INFO:root:creating /content/gdrive/MyDrive/rl_lab_2022/imitation_learning/mujoco/mujoco-py/mujoco_py/generated/_pyxbld_2.1.2.14_310_linuxcpuextensionbuilder/temp.linux-x86_64-cpython-310
INFO:root:creating /content/gdrive/MyDrive/rl_lab_2022/imitation_learning/mujoco/mujoco-py/mujoco_py/generated/_pyxbld_2.1.2.14_310_linuxcpuextensionbuilder/temp.linux-x86_64-cpython-310/content
INFO:root:creating /content/gdrive/MyDrive/rl_lab_2022/imitation_learning/mujoco/mujoco-py/mujoco_py/generated/_pyxbld_2.1.2.14_310_linuxcpuextensionbuilder/temp.linux-x86_64-cpython-310/content/gdrive
INFO:root:creating /content/gdrive/MyDrive/rl_lab_2022/imitation_learning/mujoco/mujoco-py/mujoco_py/generated/_pyxbld_2.1.2.14_310_linuxcpuextensionbuilder/temp.linux-x86_64-cpython-31

In [ ]:
#@title Download the expert checkpoint

if not os.path.exists(f'{LAB_PATH}/expert_checkpoint'):
    %cd $LAB_PATH
    !gdown --id 1CNhGwvqsLd-H0dwh-4L9rEqIo04CyOLW
    !unzip expert_checkpoint.zip

/content/gdrive/MyDrive/rl_lab_2022/imitation_learning
/usr/local/lib/python3.10/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1CNhGwvqsLd-H0dwh-4L9rEqIo04CyOLW
To: /content/gdrive/MyDrive/rl_lab_2022/imitation_learning/expert_checkpoint.zip
100% 382k/382k [00:00<00:00, 68.9MB/s]
Archive:  expert_checkpoint.zip
   creating: expert_checkpoint/
  inflating: __MACOSX/._expert_checkpoint  
   creating: expert_checkpoint/variables/
  inflating: __MACOSX/expert_checkpoint/._variables  
  inflating: expert_checkpoint/saved_model.pb  
   creating: expert_checkpoint/assets/
  inflating: __MACOSX/expert_checkpoint/._assets  
  inflating: expert_checkpoint/variables/variables.data-00000-of-00001  
  inflating: expert_checkpoint/variables/variables.index  


In [ ]:
import glob
import time

from functools import partial

import gym
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

from base64 import b64encode
from IPython.display import HTML
from pyvirtualdisplay import Display
from gym.wrappers.record_video import RecordVideo


# Start virtual display
display = Display(visible=0, size=(1024, 768))
display.start()

# Seed random generators
tf.random.set_seed(42)
np.random.seed(42)

# Helpers

def show_video(file_name):
    mp4 = open(file_name,'rb').read()
    data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
    return HTML("""
    <video width=480 controls>
        <source src="%s" type="video/mp4">
    </video>
    """ % data_url)

def mlp(input_shape, output_size, hidden_sizes=(256, 256), hidden_activation=tf.tanh, output_activation=None, l2_weight=0.0001):
    """Creates MLP with the specified parameters."""
    model = tf.keras.Sequential()

    model.add(tf.keras.Input(shape=input_shape))
    for h in hidden_sizes:
        model.add(tf.keras.layers.Dense(units=h,
                                        activation=hidden_activation,
                                        kernel_regularizer=tf.keras.regularizers.L2(l2_weight)))
    model.add(tf.keras.layers.Dense(units=output_size, activation=output_activation))

    return model

def run_policy (env, model, total_steps=10000, verbose=True):
    obs_array = np.empty([total_steps, *env.observation_space.shape])
    act_array = np.empty([total_steps, env.action_space.shape[0]])
    rew_array = np.empty([total_steps, 1])
    done_array = np.empty([total_steps, 1])

    iter_time = time.time()
    done = True
    for i in range(total_steps):
        if verbose and (i + 1) % 1000 == 0:
            steps_per_second = 1000 / (time.time() - iter_time)
            print(f'Step {i + 1}/{total_steps}, Steps per second: {steps_per_second}')
            iter_time = time.time()


        if done:
            obs = env.reset()

        act = model(tf.expand_dims(obs, axis=0))[0]
        obs_, rew, done, _ = env.step(act)
        
        obs_array[i] = obs
        act_array[i] = act
        rew_array[i] = rew
        done_array[i] = float(done)

        obs = obs_

    return obs_array, act_array, rew_array, done_array

def calculate_returns(rew, done):
    rew_cumsum = np.cumsum(rew)[:, None]
    ret_cumsum = rew_cumsum * done
    ret_cumsum_trimed = ret_cumsum[np.nonzero(ret_cumsum)]
    ret_cumsum_trimed[1:] -= ret_cumsum_trimed[:-1]
    return ret_cumsum_trimed

def evaluate_agent(env, model, verbose=False):
    _, _, rew, done = run_policy(env, model, total_steps=25000, verbose=verbose)
    rets = calculate_returns(rew, done)

    print(f'Num. episodes: {len(rets)}')
    print(f'Avg. return: {np.mean(rets)}')
    print(f'Max. return: {np.max(rets)}')
    print(f'Min. return: {np.min(rets)}')

def render_agent(env, model):
    # envw = gym.wrappers.Monitor(env, "./", force=True) # commented line
    envw = RecordVideo(env, './')
    # envw = env # added line
    o, d = envw.reset(), False
    while not d:
        envw.render()
        o, _, d, _ = envw.step(model(tf.expand_dims(o, axis=0))[0])
        # env.render() # added line
    # envw.close()

    file_name = glob.glob('rl-video-episode-*.mp4')[0]
    return show_video(file_name)

class Expert:
    """Streamlined Off-Policy (SOP) actor"""

    def __init__(self, ckpt_path):
        self.model = tf.keras.models.load_model(ckpt_path)

    def __call__(self, obs, exploratory=False):
        # We need to add one more dim. for this model
        mu, pi = self.model(tf.expand_dims(obs, axis=0))
        return pi[0] if exploratory else mu[0]

## 0. Ant

a three-dimensional quadrupedal robot.

- Observations are 111-dim. vectors that describe the kinematic properties of the robot,
- Actions are 8-dim. vectors which specify torques to be applied on the robot joints,
- The goal is to run forward as fast as possible and don’t fall over.

In [ ]:
%cd $LAB_PATH
expert = Expert('expert_checkpoint')
env = gym.make('Ant-v2',  render_mode="rgb_array")

/content/gdrive/MyDrive/rl_lab_2022/imitation_learning


/usr/local/lib/python3.10/dist-packages/gym/envs/registration.py:593: UserWarning: WARN: The environment Ant-v2 is out of date. You should consider upgrading to version `v4`.
  logger.warn(
/usr/local/lib/python3.10/dist-packages/gym/envs/mujoco/mujoco_env.py:237: UserWarning: WARN: This version of the mujoco environments depends on the mujoco-py bindings, which are no longer maintained and may stop working. Please upgrade to the v4 versions of the environments (which depend on the mujoco python bindings instead), unless you are trying to precisely replicate previous works).
  logger.warn(
/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initiali

In [ ]:
render_agent(env, expert)

In [ ]:
evaluate_agent(env, expert, verbose=True)

Step 1000/25000, Steps per second: 61.582057598718436
Step 2000/25000, Steps per second: 52.00553152936649
Step 3000/25000, Steps per second: 65.64613883515159
Step 4000/25000, Steps per second: 68.2556795463751
Step 5000/25000, Steps per second: 68.93491119397821
Step 6000/25000, Steps per second: 65.48794924049399
Step 7000/25000, Steps per second: 65.4374440520372
Step 8000/25000, Steps per second: 56.071895483466136
Step 9000/25000, Steps per second: 72.05068812524436
Step 10000/25000, Steps per second: 53.20716023103528
Step 11000/25000, Steps per second: 72.43139776784132
Step 12000/25000, Steps per second: 70.77647488267759
Step 13000/25000, Steps per second: 68.62856618514743
Step 14000/25000, Steps per second: 68.53139259437827
Step 15000/25000, Steps per second: 68.38763397273583
Step 16000/25000, Steps per second: 62.97685576106601
Step 17000/25000, Steps per second: 72.01469762552266
Step 18000/25000, Steps per second: 72.56251142897823
Step 19000/25000, Steps per second: 5

## 1. Behaviour Clonning

1. Collect the expert data.
2. Fit the model (classifier/regressor) to the expert data.

In [ ]:
# Collect the expert data
# obs, act, _, _ = run_policy(env, expert, total_steps=100000)

# Download the expert data instead of collecting it
# if not os.path.exists(f'expert_greedy_data.npz'):
#     !gdown --id 1Ny7xe3aZR-u2TpaoNhDG6wI4SS5N8FR8
# with np.load('expert_greedy_data.npz') as data:
#     obs, act, _, _ = data.values()

In [ ]:
# Use data provided by Krzysztof Jankowski

if not os.path.exists(f'expert_greedy_data.npz'):
    !gdown --id 1-0FtkebJvIZ0NUTftMyRavqkV_b0nDVl
    
with np.load('expert_greedy_data.npz') as data:
    obs, act, _, _ = data.values()

In [ ]:
# EXERCISE: Create the imitator model
input_shape = env.observation_space.shape
output_size = env.action_space.shape[0]
imitator = mlp(input_shape, output_size, hidden_sizes=(256, 256))

# We will start our experiments from the same weights for the fair comparison
init_weights = imitator.get_weights()

In [ ]:
# EXERCISE: Fit the model to the expert data
imitator.set_weights(init_weights)

# ANSWER
imitator.compile(optimizer='adam', loss='mse')
imitator.fit(obs, act, epochs=25)
# END ANSWER

evaluate_agent(env, imitator)

Epoch 1/25
3125/3125 [==============================] - 18s 5ms/step - loss: 0.0929
Epoch 2/25
3125/3125 [==============================] - 17s 5ms/step - loss: 0.0654
Epoch 3/25
3125/3125 [==============================] - 16s 5ms/step - loss: 0.0582
Epoch 4/25
3125/3125 [==============================] - 17s 5ms/step - loss: 0.0550
Epoch 5/25
3125/3125 [==============================] - 17s 5ms/step - loss: 0.0528
Epoch 6/25
3125/3125 [==============================] - 17s 5ms/step - loss: 0.0510
Epoch 7/25
3125/3125 [==============================] - 17s 6ms/step - loss: 0.0497
Epoch 8/25
3125/3125 [==============================] - 17s 5ms/step - loss: 0.0489
Epoch 9/25
3125/3125 [==============================] - 17s 5ms/step - loss: 0.0482
Epoch 10/25
3125/3125 [==============================] - 17s 5ms/step - loss: 0.0477
Epoch 11/25
3125/3125 [==============================] - 17s 5ms/step - loss: 0.0470
Epoch 12/25
3125/3125 [==============================] - 17s 5ms/step - lo

### Exercise

Discuss the questions

1. In principle, do we need the expert policy for BC?

  > Answer: No, it is sufficient to use only trajectories. However, we may incorporate it.

1. What are the problems with BC?

  > Answer: The learner's prediction affecting future observations violates the i.i.d. assumption in statistical learning, leading to poor performance. A classifier with an error probability of $\epsilon$ under expert demonstration can make up to $T^2 \epsilon$ mistakes on average over T steps due to compounding errors caused by encountering different observations.

1. How can we help BC do better?

  > Answer: Give it data that is more diverse, by for example providing data generated by an exploratory policy.

In [ ]:
# Collect the exploratory data
def exploratory(obs):
    """Adds the Gaussian noise to the expert actions."""
    act = expert(obs)
    return act + 0.29 * tf.random.normal(tf.shape(act))
# obs_expl, act_expl, rew_expl, done_expl = run_policy(env, exploratory, total_steps=100000)

# Download instead...
if not os.path.exists(f'expert_exploratory_data.npz'):
    # !gdown --id 1O-xDKmH0a8i6Q5Zfmqc6qZ0Yi8H3q5ZB # doesnt work
    !gdown --id 1-9C1hdDY7Q3ckBY3ToVF2VF-BtcoMhQN # Data provided by Krzysztof Jankowski
with np.load('expert_exploratory_data.npz') as data:
    obs_expl, act_expl, rew_expl, done_expl = data.values()

In [ ]:
# EXERCISE: Run BC on the exploratory data

imitator.set_weights(init_weights)

# ANSWER
imitator.compile(optimizer='adam', loss='mse')
imitator.fit(obs_expl, act_expl, epochs=25)
# END ANSWER

evaluate_agent(env, imitator)

Epoch 1/25
3125/3125 [==============================] - 18s 5ms/step - loss: 0.2541
Epoch 2/25
3125/3125 [==============================] - 17s 5ms/step - loss: 0.2172
Epoch 3/25
3125/3125 [==============================] - 17s 5ms/step - loss: 0.2043
Epoch 4/25
3125/3125 [==============================] - 17s 6ms/step - loss: 0.1986
Epoch 5/25
3125/3125 [==============================] - 20s 6ms/step - loss: 0.1942
Epoch 6/25
3125/3125 [==============================] - 19s 6ms/step - loss: 0.1913
Epoch 7/25
3125/3125 [==============================] - 18s 6ms/step - loss: 0.1895
Epoch 8/25
3125/3125 [==============================] - 17s 5ms/step - loss: 0.1878
Epoch 9/25
3125/3125 [==============================] - 17s 5ms/step - loss: 0.1866
Epoch 10/25
3125/3125 [==============================] - 19s 6ms/step - loss: 0.1856
Epoch 11/25
3125/3125 [==============================] - 17s 5ms/step - loss: 0.1847
Epoch 12/25
3125/3125 [==============================] - 18s 6ms/step - lo

### Exercise

Answer the questions

1. Why does it better?

  > Answer: Because it can learn to recover from errors caused by noise in exploratory actions.

1. How can we use the expert to further improve the data?

  > Hint: Noisy actions help in collecting more diverse data, but we don't want to learn the exploratory actions.

  > Answer: Infere the expert actions on the exploratory observations and run BC on it.

In [ ]:
# EXERCISE: Infere the expert actions on the exploratory observations
#           and run BC on it.

imitator.set_weights(init_weights)

# ANSWER
act_expl_infered = expert(obs_expl)

imitator.compile(optimizer='adam', loss='mse')
imitator.fit(obs_expl, act_expl_infered, epochs=25)
# ANSWER END

evaluate_agent(env, imitator)

Epoch 1/25
3125/3125 [==============================] - 17s 5ms/step - loss: 0.1614
Epoch 2/25
3125/3125 [==============================] - 16s 5ms/step - loss: 0.1244
Epoch 3/25
3125/3125 [==============================] - 16s 5ms/step - loss: 0.1127
Epoch 4/25
3125/3125 [==============================] - 19s 6ms/step - loss: 0.1075
Epoch 5/25
3125/3125 [==============================] - 16s 5ms/step - loss: 0.1041
Epoch 6/25
3125/3125 [==============================] - 16s 5ms/step - loss: 0.1017
Epoch 7/25
3125/3125 [==============================] - 16s 5ms/step - loss: 0.0998
Epoch 8/25
3125/3125 [==============================] - 16s 5ms/step - loss: 0.0981
Epoch 9/25
3125/3125 [==============================] - 16s 5ms/step - loss: 0.0969
Epoch 10/25
3125/3125 [==============================] - 16s 5ms/step - loss: 0.0958
Epoch 11/25
3125/3125 [==============================] - 16s 5ms/step - loss: 0.0949
Epoch 12/25
3125/3125 [==============================] - 16s 5ms/step - lo

### Exercise

Answer the questions

1. Did it help? Why?

  > Answer: Surprisingly, it did not help, the noise might have been averaged out in the first method.

1. How can you extend this idea?

  > Hint: How can we get more exploratory data?

  > Answer:  Use the learned cloned policy and use it to collect new data. This way it may correct its own prediction errors.

## 2. Imitation Learning via Interactive Demostrator

[DAgger](https://www.ri.cmu.edu/pub_files/2011/4/Ross-AISTATS11-NoRegret.pdf)

1. Collect the expert data.
2. Fit the model (classifier/regressor) to the expert data.
3. Collect the imitator data.
4. Infere the expert actions on the imitator data.
5. Fit the model to the extended dataset.
6. Repeat from 3.

In [ ]:
# We will pre-train on less expert data to keep the same dataset size
obs_ = obs[:30000,:]
act_ = act[:30000,:]

In [ ]:
# EXERCISE: Pretrain for 25 epochs

imitator.set_weights(init_weights)

# ANSWER
imitator.compile(optimizer='adam', loss='mse')
imitator.fit(obs_, act_, epochs=25)
# END ANSWER

evaluate_agent(env, imitator)

Epoch 1/25
938/938 [==============================] - 6s 5ms/step - loss: 0.1153
Epoch 2/25
938/938 [==============================] - 4s 4ms/step - loss: 0.0894
Epoch 3/25
938/938 [==============================] - 5s 5ms/step - loss: 0.0788
Epoch 4/25
938/938 [==============================] - 6s 7ms/step - loss: 0.0717
Epoch 5/25
938/938 [==============================] - 4s 4ms/step - loss: 0.0669
Epoch 6/25
938/938 [==============================] - 4s 4ms/step - loss: 0.0638
Epoch 7/25
938/938 [==============================] - 5s 6ms/step - loss: 0.0612
Epoch 8/25
938/938 [==============================] - 5s 6ms/step - loss: 0.0591
Epoch 9/25
938/938 [==============================] - 4s 4ms/step - loss: 0.0574
Epoch 10/25
938/938 [==============================] - 4s 4ms/step - loss: 0.0568
Epoch 11/25
938/938 [==============================] - 6s 7ms/step - loss: 0.0553
Epoch 12/25
938/938 [==============================] - 5s 5ms/step - loss: 0.0548
Epoch 13/25
938/938 [====

In [ ]:
# Exercise: Implement DAgger

for i in range(7):
    print(f'\n### Iter. {i+1} ###')

    # ANSWER
    print('\n1. Data collection')
    obs_extra, _, _, _ = run_policy(env, imitator, total_steps=10_000) # Collect 10k steps with clonedp olicy

    act_extra_infered = expert(obs_extra)  # Infere the expert actions

    # Extend the dataset
    obs_ = np.concatenate([obs_, obs_extra], axis=0)
    act_ = np.concatenate([act_, act_extra_infered], axis=0)

   
    print('\n2. Training')
    imitator.set_weights(init_weights)
    
    imitator.compile(optimizer='adam', loss='mse')
    imitator.fit(obs_, act_, epochs=25)
    # END ANSWER

    print('\n3. Evaluation')
    evaluate_agent(env, imitator)


### Iter. 1 ###

1. Data collection
Step 1000/10000, Steps per second: 183.48755334544566
Step 2000/10000, Steps per second: 271.0360840877292
Step 3000/10000, Steps per second: 271.71949696796236
Step 4000/10000, Steps per second: 235.45173451749
Step 5000/10000, Steps per second: 188.40284418264605
Step 6000/10000, Steps per second: 273.6533823690219
Step 7000/10000, Steps per second: 276.18509486902536
Step 8000/10000, Steps per second: 235.75490708731647
Step 9000/10000, Steps per second: 187.15704028265537
Step 10000/10000, Steps per second: 271.9666359899643

2. Training
Epoch 1/25
1250/1250 [==============================] - 6s 5ms/step - loss: 0.1385
Epoch 2/25
1250/1250 [==============================] - 8s 6ms/step - loss: 0.1076
Epoch 3/25
1250/1250 [==============================] - 6s 4ms/step - loss: 0.0962
Epoch 4/25
1250/1250 [==============================] - 6s 5ms/step - loss: 0.0882
Epoch 5/25
1250/1250 [==============================] - 7s 6ms/step - loss: 0.0833


In [75]:
render_agent(env, imitator)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### Note

Training the expert with the SOP algorithm (Wang et al., 2020) took 3M data samples (env. interactions). Here, we nearly match it with only 100k samples! Training from the expert can be much more efficient than reinforcement learning.